# EMG模型知识蒸馏训练 - 正确版本 V2

**时间序列事件检测任务的模型压缩**

## 目标
- Teacher: Meta官方DiscreteGesturesModule (6.5M参数, 48.66%准确度)
- Student: TimeSeriesStudent (880K参数, 7.4x压缩)
- 输出: 时间序列 `[batch, 9, 1598]` (正确！)
- 预期准确度: 30-40% (直接训练)

## 关键发现
✅ 输出格式: 时间序列而非单分类  
✅ 损失函数: BCE + Masking
✅ 评估方法: 使用Teacher的collect_metric  
✅ 任务定义: 多标签时间序列事件检测
⚠️ **蒸馏效果不佳**：直接训练(17.48%) > 蒸馏训练(11.12%)，采用直接训练方式

## 1. 环境设置与安装

In [1]:
# 检测运行环境
import os
import sys

IN_COLAB = 'google.colab' in sys.modules
print(f"运行环境: {'Google Colab' if IN_COLAB else 'Local'}")

if IN_COLAB:
    print("🔧 在Colab环境中设置...")
    # 切换到代码目录
    if not os.path.exists('/content/generic-neuromotor-interface'):
        !git clone https://github.com/facebookresearch/generic-neuromotor-interface.git
    os.chdir('/content/generic-neuromotor-interface')
    
    # 安装所有必要的依赖
    print("📦 安装依赖包...")
    !pip install -q pytorch-lightning==1.8.6 hydra-core==1.3.2 omegaconf==2.3.0
    !pip install -q h5py python-Levenshtein unidecode scipy pandas seaborn
    !pip install -q -e .
    
    print("✅ 环境设置完成")
else:
    print("✅ 本地环境")

运行环境: Google Colab
🔧 在Colab环境中设置...
Cloning into 'generic-neuromotor-interface'...
remote: Enumerating objects: 110, done.
remote: Counting objects: 100% (71/71), done.
remote: Compressing objects: 100% (58/58), done.
remote: Total 110 (delta 21), reused 18 (delta 13), pack-reused 39 (from 1)
Receiving objects: 100% (110/110), 4.46 MiB | 11.57 MiB/s, done.
Resolving deltas: 100% (24/24), done.
📦 安装依赖包...
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 800.3/800.3 kB 22.3 MB/s eta 0:00:0000:01
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 154.5/154.5 kB 18.3 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 87.2/87.2 kB 11.0 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 983.2/983.2 kB 65.7 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 153.3/153.3 kB 6.6 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 235.8/235.8 kB 18.2 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 3.2/3.2 MB 86.2 MB/s eta 0:00:00:00:01
  Preparing metadata 

In [2]:
# 挂载Google Drive（使用已下载的数据和模型）
USE_GOOGLE_DRIVE = True  # 已改为True，使用Drive中的数据

if IN_COLAB and USE_GOOGLE_DRIVE:
    from google.colab import drive
    drive.mount('/content/drive')
    
    # 数据和模型路径 - 使用Drive中已下载的内容
    DRIVE_DATA_DIR = '/content/drive/MyDrive/generic-neuromotor-data'
    DATA_PATH = f'{DRIVE_DATA_DIR}/data'
    MODELS_PATH = f'{DRIVE_DATA_DIR}/models'
    SAVE_DIR = '/content/drive/MyDrive/emg_distillation'
    
    os.makedirs(SAVE_DIR, exist_ok=True)
    print(f"✅ Drive已挂载")
    print(f"📂 数据路径: {DATA_PATH}")
    print(f"📂 模型路径: {MODELS_PATH}")
    print(f"💾 保存路径: {SAVE_DIR}")
else:
    SAVE_DIR = './logs/distillation'
    DATA_PATH = "~/emg_data"
    MODELS_PATH = "~/emg_models"
    print(f"✅ 将保存到: {SAVE_DIR}")

Mounted at /content/drive
✅ Drive已挂载
📂 数据路径: /content/drive/MyDrive/generic-neuromotor-data/data
📂 模型路径: /content/drive/MyDrive/generic-neuromotor-data/models
💾 保存路径: /content/drive/MyDrive/emg_distillation


## 2. 确认数据和Teacher模型（使用Drive中已下载的数据）

In [3]:
# 确认数据和模型（已在Drive中，不需要重新下载）
TASK = "discrete_gestures"

# DATA_PATH 和 MODELS_PATH 已在上一cell中设置
# 如果需要重新设置（本地环境）：
if not USE_GOOGLE_DRIVE:
    DATA_PATH = "~/emg_data"
    MODELS_PATH = "~/emg_models"

# 确认数据目录
print("🔍 检查数据和模型...")
data_path = os.path.expanduser(DATA_PATH) if not USE_GOOGLE_DRIVE else DATA_PATH
models_path = os.path.expanduser(MODELS_PATH) if not USE_GOOGLE_DRIVE else MODELS_PATH

# 检查数据（数据直接在 data/ 目录下，不在子目录中）
data_dir = data_path  # 直接使用 data_path，不需要添加 TASK/full_data 子路径
if os.path.exists(data_dir):
    num_files = len([f for f in os.listdir(data_dir) if f.endswith('.hdf5')])
    print(f"✅ 数据目录存在: {data_dir}")
    print(f"   包含 {num_files} 个HDF5文件")
else:
    print(f"❌ 数据目录不存在: {data_dir}")
    print("   请确认Drive中的路径是否正确")

# 检查Teacher模型（模型在 models/discrete_gestures/ 子目录下）
TEACHER_CHECKPOINT = os.path.join(models_path, TASK, "model_checkpoint.ckpt")
if os.path.exists(TEACHER_CHECKPOINT):
    size_mb = os.path.getsize(TEACHER_CHECKPOINT) / 1024 / 1024
    print(f"✅ Teacher模型存在: {TEACHER_CHECKPOINT}")
    print(f"   大小: {size_mb:.1f} MB")
else:
    print(f"❌ Teacher模型不存在: {TEACHER_CHECKPOINT}")
    print("   请确认Drive中的路径是否正确")
    # 尝试查找可能的模型文件
    if os.path.exists(models_path):
        print(f"   models/ 目录内容:")
        for f in os.listdir(models_path):
            print(f"     - {f}")

print(f"\n📍 Teacher checkpoint: {TEACHER_CHECKPOINT}")

🔍 检查数据和模型...
✅ 数据目录存在: /content/drive/MyDrive/generic-neuromotor-data/data
   包含 100 个HDF5文件
✅ Teacher模型存在: /content/drive/MyDrive/generic-neuromotor-data/models/discrete_gestures/model_checkpoint.ckpt
   大小: 74.2 MB

📍 Teacher checkpoint: /content/drive/MyDrive/generic-neuromotor-data/models/discrete_gestures/model_checkpoint.ckpt


## 3. 配置超参数

In [4]:
# 蒸馏训练超参数 - L4 GPU优化（调整后）
EXPERIMENT_ID = "distill_timeseries_v2_L4"
MAX_EPOCHS = 30  # 快速训练，可后续增加到50-100
BATCH_SIZE = 128  # L4 GPU优化（24GB显存，从64增加到128）
TEMPERATURE = 4.0  # 蒸馏温度
ALPHA = 0.3  # 蒸馏损失权重（0.3*蒸馏 + 0.7*真实标签 - 更多学习真实标签）
LEARNING_RATE = 2e-3  # 增加学习率以加快收敛

print("="*60)
print("🎯 训练配置")
print("="*60)
print(f"实验ID: {EXPERIMENT_ID}")
print(f"最大轮数: {MAX_EPOCHS}")
print(f"批大小: {BATCH_SIZE}")
print(f"蒸馏温度: {TEMPERATURE}")
print(f"蒸馏权重Alpha: {ALPHA}")
print(f"学习率: {LEARNING_RATE}")
print("="*60)

🎯 训练配置
实验ID: distill_timeseries_v2_L4
最大轮数: 30
批大小: 128
蒸馏温度: 4.0
蒸馏权重Alpha: 0.3
学习率: 0.002


## 4. 加载Teacher模型

In [5]:
# 加载Teacher模型
import torch
import pytorch_lightning as pl
from generic_neuromotor_interface.lightning import DiscreteGesturesModule

print("📦 加载Teacher模型...")

# PyTorch 2.6+ weights_only处理
original_torch_load = torch.load
def patched_torch_load(*args, **kwargs):
    kwargs['weights_only'] = False
    return original_torch_load(*args, **kwargs)
torch.load = patched_torch_load

# 加载checkpoint
teacher = DiscreteGesturesModule.load_from_checkpoint(
    TEACHER_CHECKPOINT,
    map_location='cpu'
)

# 恢复torch.load
torch.load = original_torch_load

# 移到GPU并冻结
if torch.cuda.is_available():
    teacher = teacher.cuda()
teacher.eval()
teacher.freeze()

teacher_params = sum(p.numel() for p in teacher.parameters())
print(f"✅ Teacher模型加载完成")
print(f"   参数量: {teacher_params / 1e6:.2f}M")
print(f"   网络配置: stride={teacher.network.stride}, left_context={teacher.network.left_context}")

# 测试输出形状
test_input = torch.randn(2, 16, 16000)
if torch.cuda.is_available():
    test_input = test_input.cuda()
with torch.no_grad():
    test_output = teacher(test_input)
print(f"   输出形状: {test_output.shape}  # [batch, 9, 1598]")

📦 加载Teacher模型...


/usr/local/lib/python3.12/dist-packages/pytorch_lightning/utilities/parsing.py:262: UserWarning: Attribute 'network' is an instance of `nn.Module` and is already saved during checkpointing. It is recommended to ignore them using `self.save_hyperparameters(ignore=['network'])`.
  rank_zero_warn(


✅ Teacher模型加载完成
   参数量: 6.48M
   网络配置: stride=10, left_context=20
   输出形状: torch.Size([2, 9, 1598])  # [batch, 9, 1598]


## 4.5 性能优化：复制数据到本地（可选但强烈推荐）

**问题说明**：
- 数据加载需要打开90个HDF5文件（80训练+10验证）
- 从Drive读取HDF5文件很慢（5-10分钟）
- 训练时每个epoch也要从Drive读取数据（慢20-30%）

**优化方案**：
- 一次性复制31GB数据到Colab本地SSD（5-8分钟）
- 后续加载和训练都很快（setup仅需1分钟，训练快30%）
- **总时间节省：约30分钟**

运行下面的cell来复制数据（如果已在本地环境则跳过）。

In [6]:
# 复制数据到本地以加速训练（Colab环境推荐）
import shutil
from pathlib import Path

if IN_COLAB and USE_GOOGLE_DRIVE:
    # 本地数据路径
    LOCAL_DATA_PATH = '/content/emg_data_local'
    LOCAL_MODELS_PATH = '/content/emg_models_local'
    
    # 检查是否已复制
    if not os.path.exists(LOCAL_DATA_PATH):
        print(f"📋 开始复制数据到本地SSD...")
        print(f"   源路径: {DATA_PATH}")
        print(f"   目标路径: {LOCAL_DATA_PATH}")
        print(f"   数据大小: 约31GB，预计5-8分钟")
        print(f"\n⏳ 复制中，请耐心等待...")
        
        import time
        start_time = time.time()
        
        # 复制数据文件夹
        shutil.copytree(DATA_PATH, LOCAL_DATA_PATH)
        
        # 复制模型文件夹
        shutil.copytree(MODELS_PATH, LOCAL_MODELS_PATH)
        
        elapsed = time.time() - start_time
        print(f"\n✅ 复制完成！耗时: {elapsed/60:.1f} 分钟")
        print(f"   数据: {LOCAL_DATA_PATH}")
        print(f"   模型: {LOCAL_MODELS_PATH}")
        
        # 更新路径变量
        DATA_PATH = LOCAL_DATA_PATH
        MODELS_PATH = LOCAL_MODELS_PATH
        TEACHER_CHECKPOINT = os.path.join(MODELS_PATH, TASK, "model_checkpoint.ckpt")
        
        print(f"\n💡 现在数据加载和训练都会使用本地副本（快很多！）")
    else:
        print(f"✅ 数据已在本地:")
        print(f"   {LOCAL_DATA_PATH}")
        
        # 直接使用本地路径
        DATA_PATH = LOCAL_DATA_PATH
        MODELS_PATH = LOCAL_MODELS_PATH
        TEACHER_CHECKPOINT = os.path.join(MODELS_PATH, TASK, "model_checkpoint.ckpt")
        
        print(f"💡使用本地数据，跳过复制")
else:
    print(f"⏭️  本地环境或未使用Drive，跳过复制")

📋 开始复制数据到本地SSD...
   源路径: /content/drive/MyDrive/generic-neuromotor-data/data
   目标路径: /content/emg_data_local
   数据大小: 约31GB，预计5-8分钟

⏳ 复制中，请耐心等待...

✅ 复制完成！耗时: 14.7 分钟
   数据: /content/emg_data_local
   模型: /content/emg_models_local

💡 现在数据加载和训练都会使用本地副本（快很多！）


## 5. 准备数据模块

In [7]:
# 准备数据加载器
from generic_neuromotor_interface.data_module import WindowedEmgDataModule
from generic_neuromotor_interface.data import DataSplit
from generic_neuromotor_interface.transforms import DiscreteGesturesTransform
from generic_neuromotor_interface.augmentation import RotationAugmentation
import time

print("📊 准备数据模块...")

# 数据路径处理
if USE_GOOGLE_DRIVE:
    data_location = DATA_PATH
else:
    data_location = os.path.expanduser(DATA_PATH)

# 加载数据split配置
csv_file = os.path.join(data_location, f"{TASK}_corpus.csv")
print(f"📄 加载数据split配置: {csv_file}")

data_split = DataSplit.from_csv(
    csv_filename=csv_file,
    pool_test_partitions=True
)
print(f"✅ 数据split加载完成")

# Transform和Augmentation
print(f"\n🔧 配置数据转换...")
transform = DiscreteGesturesTransform(pulse_window=(0, 8))  # 40ms at 200Hz
emg_augmentation = RotationAugmentation()

# 创建数据模块
print(f"🏗️  创建数据模块...")
data_module = WindowedEmgDataModule(
    data_location=data_location,
    data_split=data_split,
    transform=transform,
    emg_augmentation=emg_augmentation,
    window_length=16000,  # 80秒 @ 200Hz
    stride=8000,  # 50% overlap
    batch_size=BATCH_SIZE,
    num_workers=4  # L4 GPU优化：增加数据加载线程
)

# Setup数据加载器（打开100个HDF5文件）
print(f"\n⚙️  设置数据加载器（打开HDF5文件）...")
setup_start = time.time()

data_module.setup('fit')

setup_time = time.time() - setup_start
print(f"✅ Setup完成，耗时: {setup_time:.1f}秒")

print(f"\n✅ 数据模块创建完成")
print(f"   训练batches: {len(data_module.train_dataloader())}")
print(f"   验证batches: {len(data_module.val_dataloader())}")
print(f"   Batch size: {BATCH_SIZE}")

# 验证数据格式
print(f"\n📦 获取样本数据...")
sample_batch = next(iter(data_module.train_dataloader()))
print(f"✅ 数据格式检查:")
print(f"   EMG shape: {sample_batch['emg'].shape}  # [batch, channels, time]")
print(f"   Targets shape: {sample_batch['targets'].shape}  # [batch, classes, time]")

📊 准备数据模块...
📄 加载数据split配置: /content/emg_data_local/discrete_gestures_corpus.csv
✅ 数据split加载完成

🔧 配置数据转换...
🏗️  创建数据模块...

⚙️  设置数据加载器（打开HDF5文件）...


[setup] Loading datasets for split train:   0%|          | 0/80 [00:00<?, ?it/s]

[setup] Loading datasets for split val:   0%|          | 0/10 [00:00<?, ?it/s]

✅ Setup完成，耗时: 211.7秒

✅ 数据模块创建完成
   训练batches: 216
   验证batches: 27
   Batch size: 128

📦 获取样本数据...
✅ 数据格式检查:
   EMG shape: torch.Size([128, 16, 16000])  # [batch, channels, time]
   Targets shape: torch.Size([128, 9, 16000])  # [batch, classes, time]


## 5.5 验证Teacher模型准确度

在开始训练Student之前，先在验证集上测试Teacher模型的准确度，确保数据加载正确。

In [8]:
# 验证Teacher模型在验证集上的准确度
print("📊 评估Teacher模型准确度...\n")

# 创建临时trainer用于evaluation
from pytorch_lightning import Trainer

eval_trainer = Trainer(
    accelerator='auto',
    devices=1,
    logger=False,
    enable_progress_bar=True
)

# 在验证集上评估Teacher
print("⚙️  在验证集上运行Teacher模型...")
teacher_results = eval_trainer.validate(teacher, data_module, verbose=False)

print("\n" + "="*60)
print("📊 Teacher模型验证结果")
print("="*60)

result = teacher_results[0]
teacher_acc = result.get('val_accuracy', result.get('accuracy', 0)) * 100
teacher_loss = result.get('val_loss', result.get('loss', 0))

print(f"验证准确度: {teacher_acc:.2f}%")
print(f"验证损失: {teacher_loss:.4f}")
print(f"\n对比:")
print(f"  实际准确度: {teacher_acc:.2f}%")
print(f"  预期准确度: 47.42%")
print(f"  差异: {teacher_acc - 47.42:+.2f}%")

if abs(teacher_acc - 47.42) < 3.0:
    print(f"\n✅ 准确度正常，数据加载正确！")
else:
    print(f"\n⚠️  准确度差异较大，可能需要检查数据")

print("="*60)

INFO:pytorch_lightning.utilities.rank_zero:GPU available: True (cuda), used: True
INFO:pytorch_lightning.utilities.rank_zero:TPU available: False, using: 0 TPU cores
INFO:pytorch_lightning.utilities.rank_zero:IPU available: False, using: 0 IPUs
INFO:pytorch_lightning.utilities.rank_zero:HPU available: False, using: 0 HPUs


📊 评估Teacher模型准确度...

⚙️  在验证集上运行Teacher模型...


[setup] Loading datasets for split val:   0%|          | 0/10 [00:00<?, ?it/s]

INFO:pytorch_lightning.accelerators.cuda:LOCAL_RANK: 0 - CUDA_VISIBLE_DEVICES: [0]


Validation: 0it [00:00, ?it/s]


📊 Teacher模型验证结果
验证准确度: 48.65%
验证损失: 1.8168

对比:
  实际准确度: 48.65%
  预期准确度: 47.42%
  差异: +1.23%

✅ 准确度正常，数据加载正确！


## 5.6 验证Teacher模型CLER

CLER (Character-Level Error Rate) 是更全面的评估指标，考虑了分类准确度和时间精度。

**说明**：
- 验证集使用 MulticlassAccuracy（快速）
- 测试集使用 CLER（更准确，但需要整个测试集）

In [17]:
# 在测试集上评估Teacher的CLER
print("📊 评估Teacher模型CLER（测试集）...\n")

# 需要先setup test集
if not hasattr(data_module, '_test_dataset'):
    print("⚙️  准备测试数据集...")
    data_module.setup('test')
    print(f"✅ 测试集准备完成")

# 解决方案：重新加载teacher模型并移到CPU（避免cuDNN错误）
print("⚙️  重新加载Teacher模型...")
# PyTorch 2.6+ weights_only处理
original_torch_load = torch.load
def patched_torch_load(*args, **kwargs):
    kwargs['weights_only'] = False
    return original_torch_load(*args, **kwargs)
torch.load = patched_torch_load

# 重新加载checkpoint - 使用CPU避免cuDNN问题
teacher_for_test = DiscreteGesturesModule.load_from_checkpoint(
    TEACHER_CHECKPOINT,
    map_location='cpu'  # 保持在CPU上
)

# 恢复torch.load
torch.load = original_torch_load

teacher_for_test.eval()
teacher_for_test.freeze()
print("✅ Teacher模型已加载（CPU模式，避免cuDNN错误）")

# 创建测试trainer（使用CPU）
test_trainer = Trainer(
    accelerator='cpu',  # 使用CPU评估CLER
    devices=1,
    logger=False,
    enable_progress_bar=True
)

# 在测试集上运行
print("\n⚙️  在测试集上运行Teacher模型（计算CLER）...")
print("💡 使用CPU评估，速度较慢但更稳定...")
test_results = test_trainer.test(teacher_for_test, data_module, verbose=False)

print("\n" + "="*60)
print("📊 Teacher模型测试结果 (CLER)")
print("="*60)

if test_results and len(test_results) > 0:
    test_result = test_results[0]
    
    if 'test_cler' in test_result:
        teacher_cler = test_result['test_cler']
        print(f"测试集CLER: {teacher_cler:.4f}")
        print(f"\n说明:")
        print(f"  CLER = Character-Level Error Rate")
        print(f"  越低越好 (0 = 完美)")
        print(f"  考虑了分类准确度和时间精度")
    else:
        print("⚠️  未找到test_cler指标")
        print(f"可用指标: {list(test_result.keys())}")
    
    if 'test_loss' in test_result:
        print(f"\n测试集损失: {test_result['test_loss']:.4f}")
else:
    print("❌ 测试失败，未获取到结果")

print("="*60)

📊 评估Teacher模型CLER（测试集）...

⚙️  准备测试数据集...


[setup] Loading datasets for split test:   0%|          | 0/10 [00:00<?, ?it/s]

/usr/local/lib/python3.12/dist-packages/pytorch_lightning/utilities/parsing.py:262: UserWarning: Attribute 'network' is an instance of `nn.Module` and is already saved during checkpointing. It is recommended to ignore them using `self.save_hyperparameters(ignore=['network'])`.
  rank_zero_warn(
INFO:pytorch_lightning.utilities.rank_zero:GPU available: True (cuda), used: False
INFO:pytorch_lightning.utilities.rank_zero:TPU available: False, using: 0 TPU cores
INFO:pytorch_lightning.utilities.rank_zero:IPU available: False, using: 0 IPUs
INFO:pytorch_lightning.utilities.rank_zero:HPU available: False, using: 0 HPUs


✅ 测试集准备完成
⚙️  重新加载Teacher模型...
✅ Teacher模型已加载（CPU模式，避免cuDNN错误）

⚙️  在测试集上运行Teacher模型（计算CLER）...
💡 使用CPU评估，速度较慢但更稳定...


/usr/local/lib/python3.12/dist-packages/pytorch_lightning/trainer/setup.py:175: PossibleUserWarning: GPU available but not used. Set `accelerator` and `devices` using `Trainer(accelerator='gpu', devices=1)`.
  rank_zero_warn(


[setup] Loading datasets for split test:   0%|          | 0/10 [00:00<?, ?it/s]

Testing: 0it [00:00, ?it/s]

/usr/local/lib/python3.12/dist-packages/pytorch_lightning/utilities/data.py:85: UserWarning: Trying to infer the `batch_size` from an ambiguous collection. The batch size we found is 1. To avoid any miscalculations, use `self.log(..., batch_size=batch_size)`.
  warning_cache.warn(



📊 Teacher模型测试结果 (CLER)
测试集CLER: 0.1819

说明:
  CLER = Character-Level Error Rate
  越低越好 (0 = 完美)
  考虑了分类准确度和时间精度

测试集损失: 2.4291


## 6. 定义TimeSeriesStudent模型

**关键改进**: 输出时间序列 `[batch, 9, 1598]` 而非单分类

In [39]:
# TimeSeriesStudent模型定义
import torch.nn as nn
import torch.nn.functional as F

class TimeSeriesStudent(nn.Module):
    """
    时间序列Student模型 - 输出与Teacher格式一致
    
    架构:
    - Conv1d: 特征提取 + 下采样 (stride=10)
    - GRU: 时间建模 (1层, 128 hidden)
    - Linear: 投影到类别空间
    
    输入: [batch, 16, 16000]
    输出: [batch, 9, 1598]
    参数: ~140K (45x压缩)
    """
    def __init__(self, num_classes=9, input_channels=16, stride=10, left_context=20):
        super().__init__()
        self.num_classes = num_classes
        self.stride = stride
        self.left_context = left_context
        
        # 卷积编码器（模仿Teacher的第一层）- 增加通道数
        self.conv1 = nn.Conv1d(input_channels, 256, kernel_size=21, stride=stride)
        self.bn1 = nn.BatchNorm1d(256)
        self.relu = nn.ReLU()
        self.dropout = nn.Dropout(0.2)
        
        # 轻量级时间建模（GRU替代LSTM）- 增加hidden size和层数
        self.gru = nn.GRU(256, 256, num_layers=2, batch_first=True, dropout=0.2)
        
        # 输出投影
        self.projection = nn.Linear(256, num_classes)
        
    def forward(self, emg):
        """
        前向传播
        
        Args:
            emg: [batch, channels=16, sequence=16000]
        
        Returns:
            logits: [batch, num_classes=9, time_steps=1598]
        """
        # 1. 卷积特征提取 + 下采样
        x = self.conv1(emg)  # [batch, 256, time_steps]
        x = self.relu(self.bn1(x))
        x = self.dropout(x)
        
        # 2. 转置为 [batch, time, features] 用于GRU
        x = x.transpose(1, 2)
        
        # 3. GRU时间建模
        x, _ = self.gru(x)  # [batch, time, 256]
        
        # 4. 投影到类别空间
        x = self.projection(x)  # [batch, time, num_classes]
        
        # 5. 转置回 [batch, num_classes, time]
        x = x.transpose(1, 2)
        
        return x

# 创建Student模型
student = TimeSeriesStudent(
    num_classes=9,
    input_channels=16,
    stride=10,
    left_context=20
)

if torch.cuda.is_available():
    student = student.cuda()

student_params = sum(p.numel() for p in student.parameters())

print("="*60)
print("✅ Student模型创建完成")
print("="*60)
print(f"参数量: {student_params / 1e6:.2f}M")
print(f"压缩比: {teacher_params / student_params:.1f}x")

# 验证输出形状
with torch.no_grad():
    student_output = student(test_input)
    print(f"\n输出形状验证:")
    print(f"  Teacher: {test_output.shape}")
    print(f"  Student: {student_output.shape}")
    print(f"  形状匹配: {test_output.shape == student_output.shape} ✅")
print("="*60)

✅ Student模型创建完成
参数量: 0.88M
压缩比: 7.4x

输出形状验证:
  Teacher: torch.Size([2, 9, 1598])
  Student: torch.Size([2, 9, 1598])
  形状匹配: True ✅


## 7. 定义蒸馏训练模块

**关键改进**: BCE + MSE蒸馏，使用Teacher的collect_metric评估

In [44]:
# TimeSeriesDistillationModule定义
from torchmetrics.classification import MulticlassAccuracy
from generic_neuromotor_interface.constants import GestureType
from generic_neuromotor_interface.lightning import FingerStateMaskGenerator

class TimeSeriesDistillationModule(pl.LightningModule):
    """
    时间序列蒸馏训练模块
    
    损失函数:
    - 蒸馏loss: MSE(student_logits, teacher_logits) with masking
    - 真实标签loss: BCEWithLogitsLoss with masking （和Teacher一致）
    - 总loss = alpha * 蒸馏 + (1-alpha) * 真实标签
    
    评估方法:
    - 使用Teacher的collect_metric方法
    - 在事件发生时刻的时间窗口内评估准确度
    """
    def __init__(self, teacher, student, temperature, alpha, learning_rate):
        super().__init__()
        self.teacher = teacher
        self.student = student
        self.temperature = temperature
        self.alpha = alpha
        self.learning_rate = learning_rate
        
        # 损失函数
        self.bce_loss = nn.BCEWithLogitsLoss(reduction='none')
        
        # Masking generator（和Teacher一致）
        self.mask_generator = FingerStateMaskGenerator(lpad=0, rpad=7)
        
        # 评估指标
        self.val_accuracy = MulticlassAccuracy(num_classes=9)
        
        # 保存超参数
        self.save_hyperparameters(ignore=['teacher', 'student'])
        
    def forward(self, x):
        return self.student(x)
    
    def training_step(self, batch, batch_idx):
        emg = batch["emg"]
        targets = batch["targets"]
        
        # 下采样targets以匹配输出长度
        targets = targets[:, :, self.teacher.network.left_context::self.teacher.network.stride]
        
        # 生成mask（和Teacher一致）
        release_mask = self.mask_generator(targets)
        mask = torch.ones_like(targets)
        mask[:, [GestureType.index_release.value, GestureType.middle_release.value], :] = release_mask
        
        # Teacher前向传播（不计算梯度）
        with torch.no_grad():
            teacher_logits = self.teacher(emg)
        
        # Student前向传播
        student_logits = self.student(emg)
        
        # 蒸馏损失：MSE on logits (with temperature and masking)
        soft_target = teacher_logits / self.temperature
        soft_student = student_logits / self.temperature
        distill_loss = F.mse_loss(soft_student, soft_target, reduction='none')
        distill_loss = (distill_loss * mask).sum() / mask.sum()
        distill_loss = distill_loss * (self.temperature ** 2)  # 缩放回原始尺度
        
        # 真实标签损失：BCE with masking
        student_loss = self.bce_loss(student_logits, targets)
        student_loss = (student_loss * mask).sum() / mask.sum()
        
        # 总损失
        loss = self.alpha * distill_loss + (1 - self.alpha) * student_loss
        
        # 记录指标
        self.log('train/loss', loss, prog_bar=True, on_step=True, on_epoch=True)
        self.log('train/distill_loss', distill_loss, on_step=False, on_epoch=True)
        self.log('train/student_loss', student_loss, on_step=False, on_epoch=True)
        
        return loss
    
    def validation_step(self, batch, batch_idx):
        emg = batch["emg"]
        targets = batch["targets"]
        
        # 下采样targets
        targets = targets[:, :, self.teacher.network.left_context::self.teacher.network.stride]
        
        # 生成mask
        release_mask = self.mask_generator(targets)
        mask = torch.ones_like(targets)
        mask[:, [GestureType.index_release.value, GestureType.middle_release.value], :] = release_mask
        
        # Student推理
        logits = self.student(emg)
        
        # BCE loss with masking
        loss = self.bce_loss(logits, targets)
        loss = (loss * mask).sum() / mask.sum()
        
        # 使用collect_metric计算准确度
        self.collect_metric(logits, targets, phase='val')
        
        self.log('val/loss', loss, prog_bar=True, on_epoch=True, on_step=False)
        
        return loss
    
    def collect_metric(self, logits, target, phase):
        """
        复制Teacher的collect_metric方法
        在事件发生时刻的时间窗口内评估分类准确度
        """
        device = logits.device
        w_start = 10  # 50 ms at 200 Hz
        w_end = 30  # 150 ms at 200 Hz
        
        probs = torch.sigmoid(logits)
        
        # 转置为 [batch, time, classes]
        probs = probs.transpose(1, 2)
        target = target.transpose(1, 2)
        
        y = target.to(torch.int32)
        y_class = []
        y_hat_class = []
        
        for batch in range(y.shape[0]):
            y_diff = torch.diff(y[batch], axis=0)
            indices = torch.argwhere(y_diff == 1)
            
            for index in indices:
                start = max(index[0] - w_start, 0)
                end = min(index[0] + w_end, y.shape[1])
                
                y_hat = probs[batch, start:end, :]
                flattened_index = y_hat.argmax()
                _, cols = y_hat.shape
                col = flattened_index % cols
                
                y_hat_class.append(col)
                y_class.append(index[1])
        
        if len(y_class) > 0:
            y_class = torch.stack(y_class).long().to(device)
            y_hat_class = torch.stack(y_hat_class).long().to(device)
        else:
            y_class = torch.zeros(1, dtype=torch.int64, device=device)
            y_hat_class = torch.zeros(1, dtype=torch.int64, device=device)
        
        self.val_accuracy.update(y_hat_class, y_class)
        self.log(f'{phase}/accuracy', self.val_accuracy, on_step=False, on_epoch=True, prog_bar=True)
    
    def configure_optimizers(self):
        optimizer = torch.optim.AdamW(
            self.student.parameters(),
            lr=self.learning_rate,
            weight_decay=1e-4
        )
        scheduler = torch.optim.lr_scheduler.CosineAnnealingLR(
            optimizer, T_max=MAX_EPOCHS
        )
        return [optimizer], [scheduler]

# 创建训练模块（直接训练，不使用蒸馏）
# 诊断显示直接训练效果更好（17.48% vs 11.12%）

class StudentTrainingModule(pl.LightningModule):
    """直接训练Student模块（不使用蒸馏）"""
    def __init__(self, student, learning_rate=2e-3):
        super().__init__()
        self.student = student
        self.learning_rate = learning_rate
        self.bce_loss = nn.BCEWithLogitsLoss(reduction='none')
        self.mask_generator = FingerStateMaskGenerator(lpad=0, rpad=7)
        self.val_accuracy = MulticlassAccuracy(num_classes=9)
        self.save_hyperparameters(ignore=['student'])
        
    def forward(self, x):
        return self.student(x)
    
    def training_step(self, batch, batch_idx):
        emg = batch["emg"]
        targets = batch["targets"]
        targets = targets[:, :, 20::10]
        
        release_mask = self.mask_generator(targets)
        mask = torch.ones_like(targets)
        mask[:, [GestureType.index_release.value, GestureType.middle_release.value], :] = release_mask
        
        logits = self.student(emg)
        loss = self.bce_loss(logits, targets)
        loss = (loss * mask).sum() / mask.sum()
        
        self.log('train/loss', loss, prog_bar=True, on_step=True, on_epoch=True)
        return loss
    
    def validation_step(self, batch, batch_idx):
        emg = batch["emg"]
        targets = batch["targets"]
        targets = targets[:, :, 20::10]
        
        release_mask = self.mask_generator(targets)
        mask = torch.ones_like(targets)
        mask[:, [GestureType.index_release.value, GestureType.middle_release.value], :] = release_mask
        
        logits = self.student(emg)
        loss = self.bce_loss(logits, targets)
        loss = (loss * mask).sum() / mask.sum()
        
        self.collect_metric(logits, targets)
        self.log('val/loss', loss, prog_bar=True, on_epoch=True, on_step=False)
        return loss
    
    def collect_metric(self, logits, target):
        device = logits.device
        w_start = 10
        w_end = 30
        probs = torch.sigmoid(logits)
        probs = probs.transpose(1, 2)
        target = target.transpose(1, 2)
        
        y = target.to(torch.int32)
        y_class = []
        y_hat_class = []
        
        for batch in range(y.shape[0]):
            y_diff = torch.diff(y[batch], axis=0)
            indices = torch.argwhere(y_diff == 1)
            for index in indices:
                start = max(index[0] - w_start, 0)
                end = min(index[0] + w_end, y.shape[1])
                y_hat = probs[batch, start:end, :]
                flattened_index = y_hat.argmax()
                _, cols = y_hat.shape
                col = flattened_index % cols
                y_hat_class.append(col)
                y_class.append(index[1])
        
        if len(y_class) > 0:
            y_class = torch.stack(y_class).long().to(device)
            y_hat_class = torch.stack(y_hat_class).long().to(device)
        else:
            y_class = torch.zeros(1, dtype=torch.int64, device=device)
            y_hat_class = torch.zeros(1, dtype=torch.int64, device=device)
        
        self.val_accuracy.update(y_hat_class, y_class)
        self.log('val/accuracy', self.val_accuracy, on_epoch=True, on_step=False, prog_bar=True)
    
    def configure_optimizers(self):
        optimizer = torch.optim.AdamW(self.parameters(), lr=self.learning_rate, weight_decay=1e-4)
        scheduler = torch.optim.lr_scheduler.CosineAnnealingLR(optimizer, T_max=MAX_EPOCHS)
        return [optimizer], [scheduler]

# 创建训练模块
training_module = StudentTrainingModule(student, learning_rate=LEARNING_RATE)

print("✅ Student训练模块创建完成（直接训练，无蒸馏）")
print(f"   Learning Rate: {LEARNING_RATE}")
print(f"   理由：诊断测试显示直接训练(17.48%) > 蒸馏训练(11.12%)")

✅ Student训练模块创建完成（直接训练，无蒸馏）
   Learning Rate: 0.002
   理由：诊断测试显示直接训练(17.48%) > 蒸馏训练(11.12%)


## 7.5 快速测试（可选，建议运行）

**为什么需要测试？**
- 确认V2版本修复生效（避免像V1那样训练很久只有15%准确率）
- 3个epoch只需5-8分钟，能快速验证Student是否正常学习

**判断标准：**
- ✅ 3 epoch后准确度 > 25%：正常，可继续完整训练
- ⚠️ 3 epoch后准确度 15-25%：学习较慢，考虑调参
- ❌ 3 epoch后准确度 < 15%：有问题，停止检查模型

运行下面的cell进行快速测试（可选但强烈建议）。

In [42]:
# 【可选】快速测试3个epoch，验证Student能否正常学习
# 运行此cell需要5-8分钟，能避免浪费1小时训练时间

QUICK_TEST = True  # 改为False跳过测试，直接完整训练

if QUICK_TEST:
    print("🧪 快速测试：训练3个epoch验证Student学习能力\n")
    
    # 创建测试用蒸馏模块
    test_distill = TimeSeriesDistillationModule(
        teacher=teacher,
        student=student,
        temperature=TEMPERATURE,
        alpha=ALPHA,
        learning_rate=LEARNING_RATE
    )
    
    # 快速测试Trainer（只跑3个epoch）
    from pytorch_lightning import Trainer
    
    test_trainer = Trainer(
        max_epochs=3,
        accelerator='auto',
        devices=1,
        precision=16,  # PyTorch Lightning 1.8兼容格式
        enable_progress_bar=True,
        logger=False,
        benchmark=True
    )
    
    print("🚀 开始3 epoch快速测试...")
    print("="*60)
    
    # 训练
    test_trainer.fit(test_distill, data_module)
    
    # 验证
    val_results = test_trainer.validate(test_distill, data_module, verbose=False)
    val_acc = val_results[0]['val/accuracy'] * 100
    
    print("\n" + "="*60)
    print("📊 快速测试结果")
    print("="*60)
    print(f"3 epoch后Student准确度: {val_acc:.2f}%")
    print(f"Teacher准确度（对比）: 48.66%")
    print()
    
    if val_acc > 25:
        print(f"✅ 准确度 > 25% - Student学习正常！")
        print(f"💡 预期30 epoch后可达: {val_acc * 1.3:.0f}-{val_acc * 1.8:.0f}%")
        print(f"👉 可以继续运行完整训练（跳过此cell，运行后续cells）")
    elif val_acc > 15:
        print(f"⚠️  准确度 15-25% - 学习较慢")
        print(f"💡 建议调整: 降低ALPHA到0.5或增加LEARNING_RATE到1e-3")
    else:
        print(f"❌ 准确度 < 15% - 可能有问题")
        print(f"💡 建议检查: Student模型定义 (cell 18) 和损失函数 (cell 20)")
    
    print("="*60)
    
    # 清理测试对象，避免影响后续训练
    del test_distill, test_trainer, val_results
    import gc
    gc.collect()
    torch.cuda.empty_cache()
    
    print("\n💡 测试完成，现在可以：")
    print("   - 如果满意：设置 QUICK_TEST=False，继续运行后续cells")
    print("   - 如果不满意：调整超参数（cell 8），重新测试")
else:
    print("⏭️  跳过快速测试，直接进行完整训练")


INFO:pytorch_lightning.utilities.rank_zero:Using 16bit native Automatic Mixed Precision (AMP)
/usr/local/lib/python3.12/dist-packages/pytorch_lightning/plugins/precision/native_amp.py:56: FutureWarning: `torch.cuda.amp.GradScaler(args...)` is deprecated. Please use `torch.amp.GradScaler('cuda', args...)` instead.
  scaler = torch.cuda.amp.GradScaler()
INFO:pytorch_lightning.utilities.rank_zero:GPU available: True (cuda), used: True
INFO:pytorch_lightning.utilities.rank_zero:TPU available: False, using: 0 TPU cores
INFO:pytorch_lightning.utilities.rank_zero:IPU available: False, using: 0 IPUs
INFO:pytorch_lightning.utilities.rank_zero:HPU available: False, using: 0 HPUs


🧪 快速测试：训练3个epoch验证Student学习能力

🚀 开始3 epoch快速测试...


[setup] Loading datasets for split train:   0%|          | 0/80 [00:00<?, ?it/s]

[setup] Loading datasets for split val:   0%|          | 0/10 [00:00<?, ?it/s]

/usr/local/lib/python3.12/dist-packages/pytorch_lightning/callbacks/model_checkpoint.py:604: UserWarning: Checkpoint directory /content/generic-neuromotor-interface/checkpoints exists and is not empty.
  rank_zero_warn(f"Checkpoint directory {dirpath} exists and is not empty.")
INFO:pytorch_lightning.accelerators.cuda:LOCAL_RANK: 0 - CUDA_VISIBLE_DEVICES: [0]
INFO:pytorch_lightning.callbacks.model_summary:
  | Name           | Type                     | Params
------------------------------------------------------------
0 | teacher        | DiscreteGesturesModule   | 6.5 M 
1 | student        | TimeSeriesStudent        | 878 K 
2 | bce_loss       | BCEWithLogitsLoss        | 0     
3 | mask_generator | FingerStateMaskGenerator | 0     
4 | val_accuracy   | MulticlassAccuracy       | 0     
------------------------------------------------------------
878 K     Trainable params
6.5 M     Non-trainable params
7.4 M     Total params
14.723    Total estimated model params size (MB)


...

INFO:pytorch_lightning.utilities.rank_zero:`Trainer.fit` stopped: `max_epochs=3` reached.


[setup] Loading datasets for split val:   0%|          | 0/10 [00:00<?, ?it/s]

INFO:pytorch_lightning.accelerators.cuda:LOCAL_RANK: 0 - CUDA_VISIBLE_DEVICES: [0]
/usr/local/lib/python3.12/dist-packages/pytorch_lightning/trainer/connectors/data_connector.py:224: PossibleUserWarning: The dataloader, val_dataloader 0, does not have many workers which may be a bottleneck. Consider increasing the value of the `num_workers` argument` (try 12 which is the number of cpus on this machine) in the `DataLoader` init to improve performance.
  rank_zero_warn(


Validation: 0it [00:00, ?it/s]


📊 快速测试结果
3 epoch后Student准确度: 11.12%
Teacher准确度（对比）: 48.66%

❌ 准确度 < 15% - 可能有问题
💡 建议检查: Student模型定义 (cell 18) 和损失函数 (cell 20)

💡 测试完成，现在可以：
   - 如果满意：设置 QUICK_TEST=False，继续运行后续cells
   - 如果不满意：调整超参数（cell 8），重新测试


### 📋 V1 vs V2 关键区别（为什么不会再出现15%）

| 问题 | V1版本（15%失败） | V2版本（当前）| 状态 |
|------|------------------|--------------|------|
| **输出格式** | Student输出单分类 `[batch, 9]` | Student输出时间序列 `[batch, 9, 1598]` | ✅ 已修复 |
| **损失函数** | CrossEntropyLoss（单分类） | BCE + MSE（时间序列+蒸馏） | ✅ 已修复 |
| **评估方法** | 简单分类准确度 | Teacher的collect_metric（事件窗口） | ✅ 已修复 |
| **Teacher验证** | 未验证 | 已验证48.66%（cell 14） | ✅ 已验证 |
| **任务类型** | 错误理解为单分类 | 正确：多标签时间序列事件检测 | ✅ 已修复 |

**结论**：V2版本已经从根本上修复了导致15%低准确率的所有问题。上面的快速测试能进一步确认。

## 7.6 诊断测试：直接训练Student（可选）

**目的**：确认问题是在蒸馏方法上还是在Student模型设计上

- 如果直接训练准确度 > 20%：说明Student模型没问题，蒸馏方法需要改进
- 如果直接训练准确度 < 15%：说明Student模型设计有问题

运行下面的cell进行诊断（快速测试3 epoch）。

In [43]:
# 诊断测试：直接训练Student（不用蒸馏）
# 这可以确认问题是在蒸馏方法还是在Student模型设计

RUN_DIAGNOSTIC = True  # 改为False跳过

if RUN_DIAGNOSTIC:
    print("🔍 诊断测试：直接训练Student（无蒸馏）\n")
    
    # 定义直接训练模块
    class DirectStudentModule(pl.LightningModule):
        """直接训练Student（不使用蒸馏）"""
        def __init__(self, student, learning_rate=2e-3):
            super().__init__()
            self.student = student
            self.learning_rate = learning_rate
            self.bce_loss = nn.BCEWithLogitsLoss(reduction='none')
            self.mask_generator = FingerStateMaskGenerator(lpad=0, rpad=7)
            self.val_accuracy = MulticlassAccuracy(num_classes=9)
            
        def forward(self, x):
            return self.student(x)
        
        def training_step(self, batch, batch_idx):
            emg = batch["emg"]
            targets = batch["targets"]
            targets = targets[:, :, 20::10]
            
            release_mask = self.mask_generator(targets)
            mask = torch.ones_like(targets)
            mask[:, [GestureType.index_release.value, GestureType.middle_release.value], :] = release_mask
            
            logits = self.student(emg)
            loss = self.bce_loss(logits, targets)
            loss = (loss * mask).sum() / mask.sum()
            
            self.log('train/loss', loss, prog_bar=True)
            return loss
        
        def validation_step(self, batch, batch_idx):
            emg = batch["emg"]
            targets = batch["targets"]
            targets = targets[:, :, 20::10]
            
            release_mask = self.mask_generator(targets)
            mask = torch.ones_like(targets)
            mask[:, [GestureType.index_release.value, GestureType.middle_release.value], :] = release_mask
            
            logits = self.student(emg)
            loss = self.bce_loss(logits, targets)
            loss = (loss * mask).sum() / mask.sum()
            
            self.collect_metric(logits, targets)
            self.log('val/loss', loss, prog_bar=True)
            return loss
        
        def collect_metric(self, logits, target):
            device = logits.device
            w_start = 10
            w_end = 30
            probs = torch.sigmoid(logits)
            probs = probs.transpose(1, 2)
            target = target.transpose(1, 2)
            
            y = target.to(torch.int32)
            y_class = []
            y_hat_class = []
            
            for batch in range(y.shape[0]):
                y_diff = torch.diff(y[batch], axis=0)
                indices = torch.argwhere(y_diff == 1)
                for index in indices:
                    start = max(index[0] - w_start, 0)
                    end = min(index[0] + w_end, y.shape[1])
                    y_hat = probs[batch, start:end, :]
                    flattened_index = y_hat.argmax()
                    _, cols = y_hat.shape
                    col = flattened_index % cols
                    y_hat_class.append(col)
                    y_class.append(index[1])
            
            if len(y_class) > 0:
                y_class = torch.stack(y_class).long().to(device)
                y_hat_class = torch.stack(y_hat_class).long().to(device)
            else:
                y_class = torch.zeros(1, dtype=torch.int64, device=device)
                y_hat_class = torch.zeros(1, dtype=torch.int64, device=device)
            
            self.val_accuracy.update(y_hat_class, y_class)
            self.log('val/accuracy', self.val_accuracy, on_epoch=True, prog_bar=True)
        
        def configure_optimizers(self):
            optimizer = torch.optim.AdamW(self.parameters(), lr=self.learning_rate, weight_decay=1e-4)
            scheduler = torch.optim.lr_scheduler.CosineAnnealingLR(optimizer, T_max=3)
            return [optimizer], [scheduler]
    
    # 创建新的Student和模块
    diagnostic_student = TimeSeriesStudent(num_classes=9, input_channels=16, stride=10, left_context=20)
    if torch.cuda.is_available():
        diagnostic_student = diagnostic_student.cuda()
    
    direct_module = DirectStudentModule(diagnostic_student, learning_rate=2e-3)
    
    # 训练
    from pytorch_lightning import Trainer
    direct_trainer = Trainer(
        max_epochs=3,
        accelerator='auto',
        devices=1,
        precision=16,
        enable_progress_bar=True,
        logger=False,
        benchmark=True
    )
    
    print("🚀 开始直接训练Student（无蒸馏）...")
    print("="*60)
    direct_trainer.fit(direct_module, data_module)
    
    val_results = direct_trainer.validate(direct_module, data_module, verbose=False)
    direct_acc = val_results[0]['val/accuracy'] * 100
    
    print("\n" + "="*60)
    print("📊 诊断测试结果对比")
    print("="*60)
    print(f"直接训练（无蒸馏）: {direct_acc:.2f}%")
    print(f"蒸馏训练: 11.12%")
    print()
    
    if direct_acc > 20:
        print("✅ Student模型设计正常")
        print("💡 问题在蒸馏方法 - 建议：")
        print("   1. 尝试更长的训练（10-30 epochs）")
        print("   2. 或者直接用Student从头训练（不蒸馏）")
    elif direct_acc > 15:
        print("⚠️  Student学习能力一般")
        print("💡 建议：增加模型容量或调整架构")
    else:
        print("❌ Student模型设计可能有问题")
        print("💡 建议：重新设计模型架构")
    
    print("="*60)
    
    # 清理
    del diagnostic_student, direct_module, direct_trainer, val_results
    import gc
    gc.collect()
    torch.cuda.empty_cache()
else:
    print("⏭️  跳过诊断测试")

INFO:pytorch_lightning.utilities.rank_zero:Using 16bit native Automatic Mixed Precision (AMP)
/usr/local/lib/python3.12/dist-packages/pytorch_lightning/plugins/precision/native_amp.py:56: FutureWarning: `torch.cuda.amp.GradScaler(args...)` is deprecated. Please use `torch.amp.GradScaler('cuda', args...)` instead.
  scaler = torch.cuda.amp.GradScaler()
INFO:pytorch_lightning.utilities.rank_zero:GPU available: True (cuda), used: True
INFO:pytorch_lightning.utilities.rank_zero:TPU available: False, using: 0 TPU cores
INFO:pytorch_lightning.utilities.rank_zero:IPU available: False, using: 0 IPUs
INFO:pytorch_lightning.utilities.rank_zero:HPU available: False, using: 0 HPUs


🔍 诊断测试：直接训练Student（无蒸馏）

🚀 开始直接训练Student（无蒸馏）...


[setup] Loading datasets for split train:   0%|          | 0/80 [00:00<?, ?it/s]

[setup] Loading datasets for split val:   0%|          | 0/10 [00:00<?, ?it/s]

/usr/local/lib/python3.12/dist-packages/pytorch_lightning/callbacks/model_checkpoint.py:604: UserWarning: Checkpoint directory /content/generic-neuromotor-interface/checkpoints exists and is not empty.
  rank_zero_warn(f"Checkpoint directory {dirpath} exists and is not empty.")
INFO:pytorch_lightning.accelerators.cuda:LOCAL_RANK: 0 - CUDA_VISIBLE_DEVICES: [0]
INFO:pytorch_lightning.callbacks.model_summary:
  | Name           | Type                     | Params
------------------------------------------------------------
0 | student        | TimeSeriesStudent        | 878 K 
1 | bce_loss       | BCEWithLogitsLoss        | 0     
2 | mask_generator | FingerStateMaskGenerator | 0     
3 | val_accuracy   | MulticlassAccuracy       | 0     
------------------------------------------------------------
878 K     Trainable params
0         Non-trainable params
878 K     Total params
1.757     Total estimated model params size (MB)


...

INFO:pytorch_lightning.utilities.rank_zero:`Trainer.fit` stopped: `max_epochs=3` reached.


[setup] Loading datasets for split val:   0%|          | 0/10 [00:00<?, ?it/s]

INFO:pytorch_lightning.accelerators.cuda:LOCAL_RANK: 0 - CUDA_VISIBLE_DEVICES: [0]
/usr/local/lib/python3.12/dist-packages/pytorch_lightning/trainer/connectors/data_connector.py:224: PossibleUserWarning: The dataloader, val_dataloader 0, does not have many workers which may be a bottleneck. Consider increasing the value of the `num_workers` argument` (try 12 which is the number of cpus on this machine) in the `DataLoader` init to improve performance.
  rank_zero_warn(


Validation: 0it [00:00, ?it/s]


📊 诊断测试结果对比
直接训练（无蒸馏）: 17.48%
蒸馏训练: 11.12%

⚠️  Student学习能力一般
💡 建议：增加模型容量或调整架构


## 8. 配置训练器和Callbacks

In [46]:
# 配置训练器
from pytorch_lightning.callbacks import ModelCheckpoint, EarlyStopping, LearningRateMonitor
from pytorch_lightning.loggers import TensorBoardLogger

# 日志目录
log_dir = os.path.join(SAVE_DIR, EXPERIMENT_ID)
os.makedirs(log_dir, exist_ok=True)

# Callbacks
checkpoint_callback = ModelCheckpoint(
    dirpath=os.path.join(log_dir, 'checkpoints'),
    filename='epoch={epoch:02d}-val_acc={val/accuracy:.4f}',
    monitor='val/accuracy',
    mode='max',
    save_top_k=3,
    save_last=True,
    verbose=True
)

early_stop_callback = EarlyStopping(
    monitor='val/accuracy',
    patience=10,
    mode='max',
    verbose=True
)

lr_monitor = LearningRateMonitor(logging_interval='epoch')

# Logger
logger = TensorBoardLogger(
    save_dir=log_dir,
    name='tensorboard_logs'
)

# Trainer - L4 GPU优化
trainer = pl.Trainer(
    max_epochs=MAX_EPOCHS,
    accelerator='auto',
    devices=1,
    callbacks=[checkpoint_callback, early_stop_callback, lr_monitor],
    logger=logger,
    precision=16,  # 混合精度训练（L4支持Tensor Cores）
    gradient_clip_val=1.0,
    log_every_n_steps=10,
    enable_progress_bar=True,
    # L4 GPU特定优化
    benchmark=True,  # 启用cudnn benchmark加速
    deterministic=False  # 允许非确定性操作以提速
)

print("="*60)
print("✅ 训练器配置完成")
print("="*60)
print(f"日志目录: {log_dir}")
print(f"最大轮数: {MAX_EPOCHS}")
print(f"早停patience: 10")
print("="*60)
print(f"混合精度: FP16")

INFO:pytorch_lightning.utilities.rank_zero:Using 16bit native Automatic Mixed Precision (AMP)
/usr/local/lib/python3.12/dist-packages/pytorch_lightning/plugins/precision/native_amp.py:56: FutureWarning: `torch.cuda.amp.GradScaler(args...)` is deprecated. Please use `torch.amp.GradScaler('cuda', args...)` instead.
  scaler = torch.cuda.amp.GradScaler()
INFO:pytorch_lightning.utilities.rank_zero:GPU available: True (cuda), used: True
INFO:pytorch_lightning.utilities.rank_zero:TPU available: False, using: 0 TPU cores
INFO:pytorch_lightning.utilities.rank_zero:IPU available: False, using: 0 IPUs
INFO:pytorch_lightning.utilities.rank_zero:HPU available: False, using: 0 HPUs


✅ 训练器配置完成
日志目录: /content/drive/MyDrive/emg_distillation/distill_timeseries_v2_L4
最大轮数: 30
早停patience: 10
混合精度: FP16


## 9. 执行训练 🚀

In [48]:
# 开始训练
import time

print("="*60)
print("🚀 开始Student训练 - L4 GPU")
print("="*60)
print(f"Teacher准确度: 48.66% (实际验证结果)")
print(f"目标Student准确度: 30-40% (直接训练)")
print(f"GPU: L4 (24GB)")
print(f"Batch Size: {BATCH_SIZE} (优化后)")
print(f"训练方式: 直接从真实标签学习（蒸馏效果不佳，已放弃）")
print(f"预计训练时间: ~30-40分钟 (L4 GPU)")
print("="*60)

start_time = time.time()

# 训练
trainer.fit(training_module, data_module)

elapsed_time = time.time() - start_time
hours = int(elapsed_time // 3600)
minutes = int((elapsed_time % 3600) // 60)

print("\n" + "="*60)
print("✅ 训练完成！")
print("="*60)
print(f"训练时间: {hours}小时 {minutes}分钟")
print(f"最佳checkpoint: {checkpoint_callback.best_model_path}")
print(f"最佳验证准确度: {checkpoint_callback.best_model_score:.4f}")
print("="*60)

🚀 开始Student训练 - L4 GPU
Teacher准确度: 48.66% (实际验证结果)
目标Student准确度: 30-40% (直接训练)
GPU: L4 (24GB)
Batch Size: 128 (优化后)
训练方式: 直接从真实标签学习（蒸馏效果不佳，已放弃）
预计训练时间: ~30-40分钟 (L4 GPU)


[setup] Loading datasets for split train:   0%|          | 0/80 [00:00<?, ?it/s]

[setup] Loading datasets for split val:   0%|          | 0/10 [00:00<?, ?it/s]

INFO:pytorch_lightning.accelerators.cuda:LOCAL_RANK: 0 - CUDA_VISIBLE_DEVICES: [0]
INFO:pytorch_lightning.callbacks.model_summary:
  | Name           | Type                     | Params
------------------------------------------------------------
0 | student        | TimeSeriesStudent        | 878 K 
1 | bce_loss       | BCEWithLogitsLoss        | 0     
2 | mask_generator | FingerStateMaskGenerator | 0     
3 | val_accuracy   | MulticlassAccuracy       | 0     
------------------------------------------------------------
878 K     Trainable params
0         Non-trainable params
878 K     Total params
1.757     Total estimated model params size (MB)


Sanity Checking: 0it [00:00, ?it/s]

...

Validation: 0it [00:00, ?it/s]

INFO:pytorch_lightning.utilities.rank_zero:Epoch 28, global step 12499: 'val/accuracy' was not in top 3


Validation: 0it [00:00, ?it/s]

INFO:pytorch_lightning.utilities.rank_zero:Epoch 29, global step 12930: 'val/accuracy' was not in top 3
INFO:pytorch_lightning.utilities.rank_zero:`Trainer.fit` stopped: `max_epochs=30` reached.



✅ 训练完成！
训练时间: 0小时 41分钟
最佳checkpoint: /content/drive/MyDrive/emg_distillation/distill_timeseries_v2_L4/checkpoints/epoch=epoch=21-val_acc=val/accuracy=0.2149.ckpt
最佳验证准确度: 0.2149


## 10. 结果评估与对比

In [50]:
# 加载最佳模型
print("📊 评估最佳模型...\n")

best_model = StudentTrainingModule.load_from_checkpoint(
    checkpoint_callback.best_model_path,
    student=student,
    learning_rate=LEARNING_RATE
)

# 在验证集上评估
val_results = trainer.validate(best_model, data_module, verbose=False)

print("="*60)
print("📊 最终结果对比")
print("="*60)
print(f"Teacher模型:")
print(f"  参数量: {teacher_params / 1e6:.2f}M")
print(f"  验证准确度: 48.66%")
print()
print(f"Student模型 (TimeSeriesStudent - 直接训练):")
print(f"  参数量: {student_params / 1e6:.2f}M")
print(f"  验证准确度: {val_results[0]['val/accuracy']*100:.2f}%")
print(f"  验证损失: {val_results[0]['val/loss']:.4f}")
print()
print(f"压缩比: {teacher_params / student_params:.1f}x")
print(f"准确度保留: {val_results[0]['val/accuracy'] / 0.4866 * 100:.1f}%")
print(f"训练方式: 直接学习（蒸馏效果不佳）")
print("="*60)

# 保存关键信息
result_summary = {
    'experiment_id': EXPERIMENT_ID,
    'training_method': 'direct',  # 直接训练，非蒸馏
    'teacher_params': teacher_params,
    'student_params': student_params,
    'compression_ratio': teacher_params / student_params,
    'teacher_val_acc': 0.4866,
    'student_val_acc': val_results[0]['val/accuracy'] if isinstance(val_results[0]['val/accuracy'], float) else val_results[0]['val/accuracy'].item(),
    'student_val_loss': val_results[0]['val/loss'] if isinstance(val_results[0]['val/loss'], float) else val_results[0]['val/loss'].item(),
    'best_checkpoint': checkpoint_callback.best_model_path,
    'training_time_minutes': int(elapsed_time / 60),
    'note': 'Direct training performs better than distillation for this task'
}

import json
with open(os.path.join(log_dir, 'result_summary.json'), 'w') as f:
    json.dump(result_summary, f, indent=2)

print(f"\n✅ 结果已保存到: {log_dir}/result_summary.json")

📊 评估最佳模型...



[setup] Loading datasets for split val:   0%|          | 0/10 [00:00<?, ?it/s]

INFO:pytorch_lightning.accelerators.cuda:LOCAL_RANK: 0 - CUDA_VISIBLE_DEVICES: [0]
/usr/local/lib/python3.12/dist-packages/pytorch_lightning/trainer/connectors/data_connector.py:224: PossibleUserWarning: The dataloader, val_dataloader 0, does not have many workers which may be a bottleneck. Consider increasing the value of the `num_workers` argument` (try 12 which is the number of cpus on this machine) in the `DataLoader` init to improve performance.
  rank_zero_warn(


Validation: 0it [00:00, ?it/s]

📊 最终结果对比
Teacher模型:
  参数量: 6.48M
  验证准确度: 48.66%

Student模型 (TimeSeriesStudent - 直接训练):
  参数量: 0.88M
  验证准确度: 21.49%
  验证损失: 0.3219

压缩比: 7.4x
准确度保留: 44.2%
训练方式: 直接学习（蒸馏效果不佳）

✅ 结果已保存到: /content/drive/MyDrive/emg_distillation/distill_timeseries_v2_L4/result_summary.json


## 11. 导出Student模型（可选）

In [ ]:
# 导出纯Student模型（不含Training wrapper）
student_save_path = os.path.join(log_dir, 'student_model.pt')
torch.save({
    'model_state_dict': best_model.student.state_dict(),
    'model_config': {
        'num_classes': 9,
        'input_channels': 16,
        'stride': 10,
        'left_context': 20
    },
    'val_accuracy': val_results[0]['val/accuracy'].item(),
    'parameters': student_params
}, student_save_path)

print(f"✅ Student模型已导出到: {student_save_path}")
print(f"\n💡 下一步:")
print(f"   1. 量化模型: 使用PyTorch的量化工具")
print(f"   2. 转换为TFLite: 用于ESP32部署")
print(f"   3. 继续训练: 加载checkpoint训练更多epochs")

## 总结

### ✅ 完成的工作
1. **正确的任务定义**: 时间序列事件检测，输出 `[batch, 9, 1598]`
2. **正确的模型架构**: TimeSeriesStudent (140K参数, 45x压缩)
3. **正确的蒸馏方法**: BCE + MSE蒸馏，保留时间序列信息
4. **正确的评估方法**: 使用Teacher的collect_metric

### 📊 预期vs实际
- **预期准确度**: 35-45%
- **实际准确度**: (训练后查看)
- **Teacher准确度**: 47.42%

### 🎯 下一步优化方向
1. **增加训练轮数**: 50-100 epochs
2. **调整超参数**: temperature (2-6), alpha (0.5-0.9)
3. **模型架构**: 增加GRU层数或hidden size
4. **量化部署**: INT8量化 → TFLite → ESP32